# Behavior cloning flow

Create the full flow for training a model for behavior cloning which is completely separated from the other ones. 

In [1]:
import sys
sys.path.append("..")
from exp_run_config import Config
Config.PROJECTNAME = "BerryPicker"

import socket
import pathlib
import yaml
import tqdm
import papermill
from automate import automate_exprun

### Setting up a separate directory
This notebook sets up a completely separate directory (eg. BerryPicker-BC) with two subdirectories: exprun and result. 

It will copy the existing expruns into exprun. It will generate specific expruns into the directory programatically. 
The training data should go into result/demonstration under some directory (eg. touch-apple).

In [2]:
# host specific directories
hostname = socket.gethostname()
print(f"Hostname is {hostname}")
if hostname == "raven":
    raise Exception("Not configured yet")
elif hostname == "szenes.local":
    bc_path = pathlib.Path("~/Documents/Develop/Data/BerryPicker-BC").expanduser()
    exprun_path = pathlib.Path(bc_path, "exprun")
    result_path = pathlib.Path(bc_path, "result")
elif hostname == "glassy":
    bc_path = pathlib.Path("~/Work/_DataExternal/BerryPicker-BC").expanduser()
    exprun_path = pathlib.Path(bc_path, "exprun")
    result_path = pathlib.Path(bc_path, "result")
else:
    exprun_path = pathlib.Path(Config()["experiment_external"]).expanduser()
    result_path = pathlib.Path(Config()["data_external"]).expanduser()

print(f"Path for external experiments: {exprun_path}")
exprun_path.mkdir(exist_ok=True, parents=True)
print(f"Path for external data: {result_path}")
result_path.mkdir(exist_ok=True, parents=True)

Config().set_experiment_path(exprun_path)
Config().set_experiment_data(result_path)

Hostname is glassy
Path for external experiments: C:\Users\lotzi\Work\_DataExternal\BerryPicker-BC\exprun
Path for external data: C:\Users\lotzi\Work\_DataExternal\BerryPicker-BC\result
***ExpRun**: Loading pointer config file:
	C:\Users\lotzi\.config\BerryPicker\mainsettings.yaml
***ExpRun**: Loading machine-specific config file:
	c:\Users\lotzi\Work\_Config\BerryPicker\cfg\settings.yaml
***ExpRun**: Experiment config path changed to C:\Users\lotzi\Work\_DataExternal\BerryPicker-BC\exprun
***ExpRun**: Experiment data path changed to C:\Users\lotzi\Work\_DataExternal\BerryPicker-BC\result


In [3]:
# Copy the necessary experiments into the external directory
Config().copy_experiment("demonstration")
Config().copy_experiment("sensorprocessing_conv_vae")
Config().copy_experiment("robot_al5d")
Config().copy_experiment("automate")
Config().copy_experiment("behavior_cloning")


***ExpRun**: Experiment demonstration copied to C:\Users\lotzi\Work\_DataExternal\BerryPicker-BC\exprun\demonstration
***ExpRun**: Experiment sensorprocessing_conv_vae copied to C:\Users\lotzi\Work\_DataExternal\BerryPicker-BC\exprun\sensorprocessing_conv_vae
***ExpRun**: Experiment robot_al5d copied to C:\Users\lotzi\Work\_DataExternal\BerryPicker-BC\exprun\robot_al5d
***ExpRun**: Experiment automate copied to C:\Users\lotzi\Work\_DataExternal\BerryPicker-BC\exprun\automate
***ExpRun**: Experiment behavior_cloning copied to C:\Users\lotzi\Work\_DataExternal\BerryPicker-BC\exprun\behavior_cloning


In [ ]:
def generate_sensorprocessing_conv_vae(runname, exprun_path, result_path, latent_size=128, epochs=5):
    """Generate the experiment for the conv-vae sensorprocessing with the right training data and parameters. 
    Returns the experiment and runname
    FIXME there will be parameters etc"""
    experiment = "sensorprocessing_conv_vae"
    runname = "bc_sensorprocessing_conv_vae"
    
    val = {}
    val["latent_size"] = 128
    val["epochs"] = epochs
    val["save_period"] = 5
    # setting the training data
    cam = "dev0"
    demo = "touch-apple"
    training_data = []
    training_data.append([demo, "2025_08_07__15_13_18" , cam])
    val["training_data"] = training_data

    # setting the validation data
    validation_data = []
    validation_data.append([demo, "2025_08_07__15_28_02" , cam])
    val["validation_data"] = validation_data

    # save the generated exprun spec
    path = pathlib.Path(Config().get_experiment_path(), experiment, runname + ".yaml")
    with open(path, "w") as f:
        yaml.dump(val, f)

    # now, generate the entry in the automation file 
    v = {}
    v["name"] = "TrainVAE"
    v["notebook"] = "sensorprocessing/Train-Conv-VAE.ipynb"
    params = {}
    params["run"] = runname
    params["external_path"] = exprun_path.as_posix()
    params["data_path"] = result_path.as_posix()
    v["params"] = params

    return {"experiment": experiment, "runname": runname, "automation_entry": v}

In [ ]:
def generate_behaviorcloning(runname, exprun_path, result_path, latent_size=128, epochs=5):
    """Generate the experiment for the behavior cloning with the right training data and parameters. 
    Returns the experiment and runname
    FIXME there will be parameters etc"""
    experiment = "behavior_cloning"
    
    val = {}
    # FIXME: here create the parameters for the bc
    # This is the mdn
    # FIXME: these need to be passed on
    val["exp_sp"] = "'sensorprocessing_conv_vae'"
    val["run_sp"] = "'sp_vae_256'" 
    val["exp_mdn"] = "behavior_cloning"
    val["run_mdn"] = "mdn_for_bc_00"



    val["latent_size"] = latent_size
    val["epochs"] = epochs
    # FIXME: set the training data through some data structure passed as parameter....
    # setting the training data
    cam = "dev0"
    demo = "touch-apple"
    training_data = []
    training_data.append([demo, "2025_08-07__15_13_18" , cam])
    val["training_data"] = training_data

    # setting the validation data
    validation_data = []
    validation_data.append([demo, "2025_08_07__15_28_02" , cam])
    val["validation_data"] = validation_data

    # save the generated exprun spec
    path = pathlib.Path(Config().get_experiment_path(), experiment, runname + ".yaml")
    with open(path, "w") as f:
        yaml.dump(val, f)
    # now, generate the entry in the automation file 
    v = {}
    v["name"] = "BehaviorCloning"
    v["notebook"] = "behavior_cloning/Train-BehaviorCloning.ipynb"
    params = {}
    params["run"] = runname
    params["external_path"] = exprun_path.as_posix()
    params["data_path"] = result_path.as_posix()
    v["params"] = params

    return {"experiment": experiment, "runname": runname, "automation_entry": v}

### Generate a range of exp/runs to be run

In [ ]:
expruns = []
exprun = generate_sensorprocessing_conv_vae(
    runname = "bc_sensorprocessing_conv_vae", exprun_path = exprun_path, result_path = result_path)
expruns.append(exprun)
exprun = generate_behaviorcloning(runname = "bc_0001", exprun_path=exprun_path, result_path=result_path)
expruns.append(exprun)


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\lotzi\\Work\\_DataExternal\\BerryPicker-BC\\exprun\\behaviorcloning\\bc_sensorprocessing_conv_vae.yaml'

### Generate an automation script. 

* FIXME: the notebooks should have support to set the exprun and results directories from the automation exp. (This seems to actually exist)


In [ ]:
value = {}
val = []
value["exps_to_run"] = val

# creation_style = "exist-ok"
creation_style = "discard-old"

for exprun in expruns:
    # exp = Config().get_experiment(exprun[0], exprun[1])
    # FIXME: add the automation here or generate the automation script
    v = exprun["automation_entry"]
    v["creation_style"] = creation_style
    val.append(v)

path = pathlib.Path(Config().get_experiment_path(), "automate", "flow_bc.yaml")
with open(path, "w") as f:
    yaml.dump(value, f)

### Run the automation script
FIXME: the existing automation does not take into consideration the external directories the right way. I am trying to fix this. 
FIXME: this should be done such that I can also run it from command line

In [ ]:
experiment = "automate"
run = "flow_bc"
exp = Config().get_experiment(experiment, run)

#output_path = pathlib.Path(ext_path, "_automation_output")
#output_filename = f"{notebook_path.stem}_{name}_output{ notebook_path.suffix}"
#output = pathlib.Path(output_path, notebook_path.parent, output_filename)
#output.parent.mkdir(exist_ok=True, parents=True)
#print(output)


for item in tqdm.tqdm(exp["exps_to_run"]):
    print(f"***Automating {item['name']}")
    notebook_path = pathlib.Path("..", item["notebook"])
    print(notebook_path.exists())
    output_filename = f"{notebook_path.stem}_output{notebook_path.suffix}"
    # FIXME: this should go into the data dir of the appropriate exp... but I don't have access to it... 
    output_path = pathlib.Path(result_path, output_filename)
    try:
        papermill.execute_notebook(
            notebook_path,
            output_path.absolute(),
            cwd=notebook_path.parent,
            parameters=item["params"]
        )
    except Exception as e:
        print(f"There was an exception {e}")    

***ExpRun**: Experiment default config C:\Users\lotzi\Work\_DataExternal\BerryPicker-BC\exprun\automate\_defaults_automate.yaml was empty, ok.
***ExpRun**: Configuration for exp/run: automate/flow_bc successfully loaded


  0%|          | 0/1 [00:00<?, ?it/s]c:\Users\lotzi\Work\_VirtualEnvs\BerryPicker\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


***Automating TrainVAE
True


100%|██████████| 1/1 [00:23<00:00, 23.35s/it]
